In [ ]:
!pip install nltk

In [ ]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 22.8 MB/s eta 0:00:00


In [ ]:
!pip install transformers
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import urllib.request
import urllib.parse
import json
import pandas as pd
from google.colab import files
import re

In [ ]:
# Naver API 클라이언트 ID와 클라이언트 시크릿
client_id = "mFUbB8A2Nto9jRgfWx7s"
client_secret = "DsEr2rzsCf"  # 여기에 실제 클라이언트 시크릿을 입력하세요

# 사용자로부터 검색어 입력 받기
query = input("검색어를 입력하세요: ")

# 검색어 인코딩
encText = urllib.parse.quote(query)

def sanitize_filename(name):
    # 공백은 밑줄로, 특수 문자는 제거
    return re.sub(r'[^\w\s-]', '', name).strip().replace(' ', '_')

filename = sanitize_filename(query) + '.csv'

# 필터링할 단어 설정
excluded_words = ['봉사','주가']  # 제외할 단어 리스트

def contains_excluded_words(text, excluded_words):
    """제목 또는 본문에 제외할 단어가 포함되어 있는지 확인"""
    return any(word in text for word in excluded_words)

# API 요청 함수
def get_search_results(start, display):
    url = f"https://openapi.naver.com/v1/search/news?query={encText}&start={start}&display={display}"
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id", client_id)
    request.add_header("X-Naver-Client-Secret", client_secret)

    response = urllib.request.urlopen(request)
    if response.getcode() == 200:
        response_body = response.read()
        return json.loads(response_body.decode('utf-8'))
    else:
        raise Exception(f"Error Code: {response.getcode()}")


# 전체 결과 저장 리스트
all_items = []

# 한번에 가져올 결과 개수
display_count = 100

# 총 결과 개수 설정 (예: 1000개의 결과를 가져옴)
total_results = 150

for start in range(1, total_results + 1, display_count):
    result = get_search_results(start, display_count)
    filtered_items = [item for item in result['items']
                      if not (contains_excluded_words(item['title'], excluded_words) or
                              contains_excluded_words(item['description'], excluded_words))]
    all_items.extend(filtered_items)
    print(f"Fetched results from {start} to {start + display_count - 1}")


# 결과를 데이터프레임으로 변환
df = pd.DataFrame(all_items)

# 데이터프레임을 CSV 파일로 저장
df.to_csv(filename, index=False, encoding='utf-8-sig')
files.download(filename)

검색어를 입력하세요: 삼보모터스
Fetched results from 1 to 100
Fetched results from 101 to 200


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt

In [ ]:
# Okt를 사용한 한글 토크나이저
class KoreanTfidfVectorizer(TfidfVectorizer):
    def __init__(self, tokenizer=None, **kwargs):
        if tokenizer is None:
            tokenizer = KoreanTokenizer()
        super().__init__(tokenizer=tokenizer, **kwargs)

class KoreanTokenizer:
    def __init__(self):
        self.okt = Okt()

    def __call__(self, text):
        return self.okt.nouns(text)

# 기사 내용 텍스트를 벡터화
vectorizer = KoreanTfidfVectorizer(stop_words=None)  # 한글의 경우 stop_words=None으로 설정
X = vectorizer.fit_transform(df['description'])

# 코사인 유사도 계산
cosine_sim = cosine_similarity(X, X)

# 중복 여부 판단 함수
def is_duplicate(index, threshold=0.8):
    # 특정 기사의 유사도 벡터를 가져옴
    similarity_scores = cosine_sim[index]
    # 유사도가 임계값(threshold)을 초과하는 다른 기사들을 찾음
    return any(score > threshold for i, score in enumerate(similarity_scores) if i != index)

# 중복된 기사 인덱스 찾기
duplicate_indices = [i for i in range(len(df)) if is_duplicate(i)]

# 중복된 기사 제거
df_cleaned = df.drop(duplicate_indices).reset_index(drop=True)

# 결과 출력
print(f"총 {len(df)}개의 기사 중 {len(duplicate_indices)}개가 중복으로 판단되어 제거되었습니다.")
print(f"중복 제거 후 {len(df_cleaned)}개의 기사가 남았습니다.")

# 중복 제거된 데이터프레임을 CSV 파일로 저장
filename_cleaned = sanitize_filename(query) + '_cleaned.csv'
df_cleaned.to_csv(filename_cleaned, index=False, encoding='utf-8-sig')

print(f"중복 제거된 결과가 '{filename_cleaned}' 파일에 저장되었습니다.")
files.download(filename_cleaned)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


총 153개의 기사 중 86개가 중복으로 판단되어 제거되었습니다.
중복 제거 후 67개의 기사가 남았습니다.
중복 제거된 결과가 '삼보모터스_cleaned.csv' 파일에 저장되었습니다.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pip install konlpy scikit-learn

In [ ]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 8.3 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/e9t/nsmc.git

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Counting objects: 100% (14762/14762), done.
remote: Compressing objects: 100% (13012/13012), done.
remote: Total 14763 (delta 1748), reused 14762 (delta 1748), pack-reused 1 (from 1)
Receiving objects: 100% (14763/14763), 56.19 MiB | 20.48 MiB/s, done.
Resolving deltas: 100% (1748/1748), done.
Updating files: 100% (14737/14737), done.


In [ ]:
!ls nsmc

code  ratings_test.txt	ratings_train.txt  ratings.txt	raw  README.md	synopses.json


In [ ]:
!cp -r nsmc /content/drive/MyDrive/

In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader

# CSV 파일 로드
train_df = pd.read_csv('/content/drive/MyDrive/nsmc/ratings_train.txt', sep='\t')
test_df = pd.read_csv('/content/drive/MyDrive/nsmc/ratings_test.txt', sep='\t')

# 데이터 전처리
# 'document' 컬럼의 결측값을 빈 문자열로 대체하고, 모든 값을 문자열로 변환
train_df['document'] = train_df['document'].fillna('').astype(str)
test_df['document'] = test_df['document'].fillna('').astype(str)

# 데이터 분리
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['document'].tolist(),
    train_df['label'].tolist(),
    test_size=0.1,
    random_state=42
)

In [ ]:
# Tokenizer 및 모델 로드
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# PyTorch 데이터셋 정의
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # Ensure that text is a string
        if not isinstance(text, str):
            raise ValueError(f"Expected text to be a string, got {type(text)}")
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        # Convert tensors to proper format
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
# 데이터셋 및 데이터로더 준비
max_length = 128
train_dataset = SentimentDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = SentimentDataset(val_texts, val_labels, tokenizer, max_length)
test_dataset = SentimentDataset(test_df['document'].tolist(), test_df['label'].tolist(), tokenizer, max_length)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [ ]:
# TrainingArguments 설정
training_args = TrainingArguments(
    output_dir='./results',  # 결과 저장 디렉토리
    evaluation_strategy='epoch',  # 평가 전략 설정 (매 epoch마다 평가)
    save_strategy='epoch',  # 저장 전략 설정 (매 epoch마다 저장)
    per_device_train_batch_size=16,  # 훈련 배치 사이즈
    per_device_eval_batch_size=16,  # 평가 배치 사이즈
    num_train_epochs=3,  # 훈련 epoch 수
    logging_dir='./logs',  # 로그 저장 디렉토리
    load_best_model_at_end=True,  # 최상의 모델을 끝에서 로드할지 여부
    metric_for_best_model='accuracy'  # 최상의 모델을 선택할 때 사용할 지표
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

# 평가 지표 함수 정의
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

In [ ]:
# Trainer 객체 생성 시 compute_metrics 전달
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics  # 평가 지표 함수 전달
)

In [ ]:
# 모델 학습
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.695600,0.695715,0.502533,0.502533,1.000000,0.668915
2,0.690600,0.695004,0.533933,0.539921,0.490714,0.514143
3,0.693900,0.697646,0.502533,0.502533,1.000000,0.668915


TrainOutput(global_step=25314, training_loss=0.6944328266513605, metrics={'train_runtime': 9072.1239, 'train_samples_per_second': 44.642, 'train_steps_per_second': 2.79, 'total_flos': 2.66399943552e+16, 'train_loss': 0.6944328266513605, 'epoch': 3.0})

In [ ]:
# 모델 저장
model.save_pretrained('/content/drive/MyDrive/saved_model')
tokenizer.save_pretrained('/content/drive/MyDrive/saved_model')

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 모델과 토크나이저 로드
model_path = '/content/drive/MyDrive/saved_model'
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
def analyze_sentiment(text):
    # 모델 입력으로 변환
    inputs = tokenizer(text, return_tensors='pt', max_length=128, truncation=True, padding=True)

    # 모델 추론
    with torch.no_grad():
        outputs = model(**inputs)

    # 로짓을 확률로 변환
    logits = outputs.logits
    probs = torch.nn.functional.softmax(logits, dim=-1)

    # 긍정과 부정 확률 추출
    prob_neg = probs[0][0].item()  # 부정 확률
    prob_pos = probs[0][1].item()  # 긍정 확률

    # 확률 임계값 설정
    threshold_neg = 0.5

    # 감성 예측
    if prob_neg >= threshold_neg:
        sentiment_label = '부정'
    else:
        sentiment_label = '긍정'

    return sentiment_label

In [ ]:
!pip install torch

In [ ]:
import torch

# Google 드라이브에서 CSV 파일 로드
file_path = '/content/drive/MyDrive/소부장 공모전/삼보모터스_cleaned.csv'
df = pd.read_csv(file_path)

# 기사 제목과 본문 감성 분석
# df['title_sentiment'] = df['title'].apply(analyze_sentiment)
df['content_sentiment'] = df['description'].apply(analyze_sentiment)

# 결과를 새 CSV 파일로 저장
output_filename = 'search_results_with_sentiment.csv'
output_path = f'/content/drive/MyDrive/{output_filename}'
df.to_csv(output_path, index=False, encoding='utf-8-sig')

print(f"감성 분석 결과가 '{output_filename}' 파일에 저장되었습니다.")
files.download(output_path)

감성 분석 결과가 'search_results_with_sentiment.csv' 파일에 저장되었습니다.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df['content_sentiment'].value_counts()

,count
content_sentiment,
긍정,38
부정,29


In [ ]:
sentiment_df = pd.read_csv('/content/drive/MyDrive/소부장 공모전/search_results_with_sentiment_0.45.csv')
subset_df = sentiment_df.head(40)
subset_df['content_sentiment'].value_counts()

,count
content_sentiment,
긍정,27
부정,13
